In [2]:
import nibabel                      as nib
import nibabel.freesurfer.mghformat as mgh

In [5]:
# MGH/MGZ files
mgh_file = mgh.load('Scans/OAS30001_MR_d0129.mgz')
mgh_file.get_fdata().shape

(256, 256, 256)

In [7]:
type(mgh_file.get_fdata())

numpy.ndarray

In [13]:
from scipy import ndimage

def read_mgz_file(filepath):
    """Read and load volume"""
    # Read file
    scan = mgh.load(filepath)
    # Get raw data
    scan = scan.get_fdata()
    return scan

def normalize(volume):
    """Normalize the volume"""
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume

#Not going to use this resizing function
def resize_volume(img):
#     """Resize across z-axis"""
#     # Set the desired depth
#     desired_depth = 64
#     desired_width = 128
#     desired_height = 128
#     # Get current depth
#     current_depth = img.shape[-1]
#     current_width = img.shape[0]
#     current_height = img.shape[1]
#     # Compute depth factor
#     depth = current_depth / desired_depth
#     width = current_width / desired_width
#     height = current_height / desired_height
#     depth_factor = 1 / depth
#     width_factor = 1 / width
#     height_factor = 1 / height
    # Rotate
    img = ndimage.rotate(img, 90, reshape=False)
#     # Resize across z-axis
#     img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
#     return img

def process_scan(path):
    """Read and resize volume"""
    # Read scan
    volume = read_mgz_file(path)
    # Normalize
#     volume = normalize(volume)
#     # Resize width, height and depth
    #Rotate 
    volume = resize_volume(volume)
    return volume

In [15]:
scan = process_scan('Scans/OAS30001_MR_d0129.mgz')

In [25]:
import matplotlib.pyplot as plt
import numpy as np

# data = scan
# images, labels = list(data)[0]
# images = images.numpy()
image = scan
print("Dimension of the CT scan is:", image.shape)
plt.imshow(np.squeeze(image[:, :, 110]), cmap="gray")

AttributeError: 'NoneType' object has no attribute 'shape'